In [ ]:
import pyOperator

class simpleChain(pyOperator.Operator):
    """ d = Op1 op2 m"""
    def __init__(self,op1,op2):
        self._op1=op1
        self._op2=op2
        if not self._op1.getDomain().checkSame(self._op2.getRange()):
            raise Exception("domain of op1 must be the same space range of op 2")
        super().setDomainRange(self._op2.getDomain(),self._op1.getRange())
        self._tmp=self._op1.getDomain().clone()
    
    def forward(self,add,mod,dat):
        self._op2.forward(False,mod,self._tmp)
        self._op1.forward(add,self._tmp,dat)
    
    def adjoint(self,add,mod,dat):
        self._op1.adjoint(False,self._tmp,dat)
        self._op2.adjoint(add,mod,self._tmp)


In [ ]:
import giee
import Hypercube
import Stack
import Dots
import Nmo
import holoviews as hv
import pyProblem
import pyStopper
import pyLinearSolver




mod=giee.vector(Hypercube.hypercube(ns=[128],os=[.0],ds=[.032]))
m=mod.getNdArray()
m[15:23]=1
m[75:83]=1
plotm=Dots.dots(mod,width=800,height=100,title="Model",dots=False,curve=True,yaxis=None,xaxis=None)


data=giee.vector(Hypercube.hypercube(ns=[128,11],os=[0,0],ds=[.032,.6]))
tmp=data.clone()
stackOp=Stack.stack_2D_2(mod,tmp)
nmoOp=Nmo.nmo2D(1.5,tmp,data)


comboOp=simpleChain(nmoOp,stackOp)
comboOp.forward(False,mod,data)
mod.zero()

problemNmoStack=pyProblem.ProblemL2Linear(mod,data,comboOp)
problemStop=pyStopper.BasicStopper(niter=11)

solve=pyLinearSolver.LCGsolver(problemStop)
solve.setDefaults(save_model=True)
solve.run(problemNmoStack)


print("where")
plotd=Dots.dots(data,width=800,height=600,title="Synthetic Data",dots=False,curve=True,xaxis=None,ylim=(0,6))
ans=[]
images=[]
for i in range(1,len(solve.modelSet.vecSet),2):
    ans.append(Dots.dots(solve.modelSet.vecSet[i],width=800,height=100,title="Iter "+str(i),dots=False,curve=True,yaxis=None))
    images.append(ans[len(ans)-1].image())




hv.Layout(plotm.image()+plotd.image()+images[0]+images[1]+images[2]+images[3]).cols(1)

